In [ ]:
import os
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from dotenv import load_dotenv
load_dotenv()


In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
loader = DirectoryLoader("data/eval_documents")
documents = loader.load()

In [ ]:
# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# generate testset
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

In [ ]:
df = testset.to_pandas()
df.to_parquet("data/eval_questions.parquet")
df.to_csv("data/eval_questions.csv")
df

In [ ]:
df.iloc[0]["question"]

In [ ]:
df.iloc[0]["ground_truth"]

In [ ]:
import requests
import json

def generate_text(prompt):
    url = "http://localhost:11434/api/generate"
    headers = {
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama3",
        "prompt": prompt,
        "parameters": {
            "max_length": 100
        }
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    if response.status_code == 200:
        response_text = response.text
        # data = json.loads(response_text)
        # actual_response = data["response"]
        return response_text
    else:
        return f"Error: {response.status_code}, {response.text}"

# Example usage
prompt = "Explain the concept of neural networks."
generated_text = generate_text(prompt)
print(generated_text)